# Automatic Removal of "Garbage Strings" in OCR Text: An Implementation

This notebook is a python implementation of the Paper `Automatic Removal of "Garbage Strings" in OCR Text: An Implementation` 

The paper describes certain rules to identify "Garbage Strings". 

Apart from the rules mentioned in the original paper, I have implemented a few more rules which I have found important.

Link to the original paper: [10.1.1.81.8901](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.81.8901&rep=rep1&type=pdf)

In [1]:
import re


class GarbageStringDetector(object):
    def __init__(self, rules=None):
        self.rules = list("LARVPC") if rules is None else rules

    def too_long(self, string):
        """
        Rule L
        ======
        tests whether the string passed is more than 40 characters, if
        yes. It is garbage!
        :param string: string to be tested
        :returns: either True or False
        """
        return True if max(list(map(len, string.split()))) > 40 else False

    def bad_alnum_ratio(self, string):
        """
        Rule A
        ======
        if a string's ratio of alphanumeric characters to total characters is
        less than 50%, the string is garbage
        :param string: string to be tested
        :returns: either True or False
        """

        pattern = re.compile("[\W_]+")
        alnum_thresholds = {
            1: 0,     # single chars can be non-alphanumeric
            2: 0,     # so can doublets
            3: 0.32,  # at least one of three should be alnum
            4: 0.24,  # at least one of four should be alnum
            5: 0.39,  # at least two of five should be alnum
        } 

        threshold = (
            alnum_thresholds[len(string)] if len(string) in alnum_thresholds else 0.5
        )

        if len(string) == 0:
            return True
        return bool(float(len(pattern.sub("", string))) / len(string) < threshold)

    def consecutive_four_identical(self, string):
        """
        Rule R
        ======
        if a string has 4 identical characters in a row, it is garbage
        :param string: string to be tested
        :returns: either True or False
        """
        pattern = re.compile(
            r"(([A-Za-z])\2{3,})"
        )  # matches any 4 consecutive characters
        return True if pattern.search(string) else False

    def bad_consonant_vowel_ratio(self, string):
        """
        Rule V
        ======
        if a string has nothing but alphabetic characters, look at the
        number of consonants and vowels. If the number of one is less than 10%
        of the number of the other, then the string is garbage.
        This includes a length threshold.
        :param string: string to be tested
        :returns: either True or False
        """
        alpha_string = list(filter(str.isalpha, string))
        vowel_count = sum(1 for char in alpha_string if char in "aeiouAEIOU")
        consonant_count = len(alpha_string) - vowel_count

        if consonant_count > 0 and vowel_count > 0:
            ratio = float(vowel_count) / consonant_count
            if ratio < 0.1 or ratio > 10:
                return True
        elif vowel_count == 0 and consonant_count > len("rhythms"):
            return True
        elif consonant_count == 0 and vowel_count > len("IEEE"):
            return True

        return False

    def has_two_distinct_puncts_inside(self, string):
        """
        Rule P
        ======
        Strip off the first and last characters of a string. If there
        are two distinct punctuation characters in the result, then the string
        is garbage
        Customisation
        =============
        stripping off the last TWO characters as false positives
        included those ending with ').' and similar.
        :param string: string to be tested
        :returns: either True or False
        """
        for token in string.split():
            non_alnum_string = "".join(
                char for char in token[1:-2] if not char.isalnum()
            )
            for char in non_alnum_string[1:]:
                if char != non_alnum_string[0]:
                    return True
        return False

    def has_uppercase_within_lowercase(self, string):
        """
        Rule C
        ======
        If a string begins and ends with a lowercase letter, then if
        the string contains an uppercase letter anywhere in between, then it
        is removed as garbage.
        Customisation
        =============
        false positive on "needed.The". Exclude fullstop-capital.
        Extra customisation: Exclude hyphen-capital, apostrophe-capital and
        forwardslash-capital
        :param string: string to be tested
        :returns: either True or False
        """
        if string and string[0].islower() and string[-1].islower():
            string_middle = string[1:-1]
            for index, char in enumerate(string_middle):
                if char.isupper() and not (
                    index > 0 and string_middle[index - 1] in ".-'"
                ):
                    return True
        return False

    def low_avg_token_len(self, string):
        """
        Rule M (not mentioned in the original paper)
        ======
        tests whether the string passed has average token length greater than 3, if
        no. It is garbage!
        :param string: string to be tested
        :returns: either True or False
        """
        return bool(sum(list(map(len, string.split()))) / len(string.split()) <= 3)

    def __call__(self, string):
        """
        passes the string to check for every rule and if any match is found
        , it returns that rule
        :param string: string to be tested
        :returns: either matched rule or False
        """

        if "L" in self.rules and self.too_long(string):
            return "L"
        if "A" in self.rules and self.bad_alnum_ratio(string):
            return "A"
        if "R" in self.rules and self.consecutive_four_identical(string):
            return "R"
        if "V" in self.rules and self.bad_consonant_vowel_ratio(string):
            return "V"
        if "P" in self.rules and self.has_two_distinct_puncts_inside(string):
            return "P"
        if "C" in self.rules and self.has_uppercase_within_lowercase(string):
            return "C"
        if "M" in self.rules and self.low_avg_token_len(string):
            return "M"
        return False


In [2]:
gsd = GarbageStringDetector()
for token in "This document was scanned perfectly".split():
    print(f"{token:30} {gsd(token)}")

This                           False
document                       False
was                            False
scanned                        False
perfectly                      False


In [3]:
for token in "This 3ccm@nt w&s scnnnnd not pe&;c1!y".split():
    print(f"{token:30} {gsd(token)}")

This                           False
3ccm@nt                        False
w&s                            False
scnnnnd                        R
not                            False
pe&;c1!y                       P
